In [14]:
# it is good practice to import all your libraries in one cell
# but I have broken it out to guide you

import pandas as pd # you will need this to use the read_sql function

# if you want to supress any warnings use this
import warnings
warnings.filterwarnings('ignore')

In [15]:
# you will need this code to import the passwords and other access information from the .env file
import os
from dotenv import load_dotenv
load_dotenv()

dbname = os.getenv("dbname")
host = os.getenv("host")
port = os.getenv("port")
user = os.getenv("user")
password = os.getenv("password")

In [16]:
# check it worked
print(user)

admin


In [17]:
# optional step
# you will need this code if you want to import the function connect_to_redshift from your python file in src folder
# into your jupyter notebooks

import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.connect_to_redshift import connect_to_redshift

In [18]:
# connect to redshift - by import the definition from the src foldr
# otherwise just copy the function and import the library psycopg2
connect = connect_to_redshift(dbname, host, port, user, password)

connection to redshift made


In [19]:
# sample sql query to get customer data from the online_transaction_cleaned table with their average order value 
# and number of unique items they purchased

query = """select customer_id,
                  country,
                  count(distinct stock_code) as number_unique_items,
                  round(avg(total_order_value), 2) as avg_order_value
           from bootcamp.online_transactions_cleaned ot 
           group by customer_id, country
           order by avg_order_value desc
        """

sample_customer_data = pd.read_sql(query, connect)

In [20]:
sample_customer_data.shape

(4371, 4)

In [21]:
# top 5 customer based on average order value
sample_customer_data.head()

,customer_id,country,number_unique_items,avg_order_value
0,u15098,United Kingdom,1,9904.88
1,u15195,United Kingdom,1,3861.00
2,u13135,United Kingdom,1,3096.00
3,u16532,United Kingdom,2,1687.20
4,u15749,United Kingdom,6,1435.73


In [22]:
# lowest 5 customers
# aah.. data quality error of < 0, could remove these from the query above

sample_customer_data.tail()

,customer_id,country,number_unique_items,avg_order_value
4366,u13829,United Kingdom,1,-102.00
4367,u12666,Israel,2,-113.72
4368,u17307,United Kingdom,1,-152.64
4369,u16742,United Kingdom,3,-154.97
4370,u14213,United Kingdom,5,-238.44


In [24]:
# sample sql query to get customer data from the online_transaction_cleaned table with their average order value 
# and number of unique items they purchased

query = """select customer_id,
                  country,
                  count(distinct stock_code) as number_unique_items,
                  round(avg(total_order_value), 2) as avg_order_value
           from bootcamp.online_transactions_cleaned ot 
           group by customer_id, country
           having avg_order_value > 0
           order by avg_order_value desc
        """

sample_customer_data = pd.read_sql(query, connect)

In [29]:
# the negative values impacted approx.
print("# of customers with negative average order values: ", 4371 - sample_customer_data.shape[0] )

# of customers with negative average order values:  43


In [30]:
# top 5 customer based on average order value
sample_customer_data.head()

,customer_id,country,number_unique_items,avg_order_value
0,u12431,Australia,117,29.06
1,u12838,United Kingdom,115,5.60
2,u12748,United Kingdom,1767,6.72
3,u14001,United Kingdom,34,52.39
4,u17968,United Kingdom,74,3.27


In [31]:
# lowest 5 customers
# aah.. data quality error of < 0, could remove these from the query above

sample_customer_data.tail()

,customer_id,country,number_unique_items,avg_order_value
4323,u12587,Finland,3,34.67
4324,u15269,United Kingdom,2,204.40
4325,u16737,United Kingdom,1,417.60
4326,u14785,United Kingdom,2,25.80
4327,u1312,United Kingdom,1,30.60
